In [ ]:
# Cell 1: Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from mealpy.swarm_based.WOA import OriginalWOA
from mealpy.utils.problem import Problem
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
# Cell 2: StableFeatureSelector class definition
class StableFeatureSelector:
    """
    A feature selector that prioritizes both accuracy AND feature stability.
    
    Feature Stability = How consistently the same features are selected across 
    different data splits/runs. High stability means robust, reliable features.
    """
    
    def __init__(self, target_features=15, stability_runs=5):
        self.target_features = target_features
        self.stability_runs = stability_runs
        self.feature_stability_scores = {}
        self.all_run_results = []
        
    def calculate_stability_metrics(self, all_selected_features):
        """
        Calculate various stability metrics for feature selection
        
        Args:
            all_selected_features: List of sets, each containing selected feature indices
        
        Returns:
            stability_score: Overall stability score (0-1, higher is better)
            feature_frequencies: How often each feature was selected
        """
        # Count how many times each feature was selected
        feature_counter = Counter()
        total_selections = len(all_selected_features)
        
        for feature_set in all_selected_features:
            for feature in feature_set:
                feature_counter[feature] += 1
        
        # Calculate frequency of each feature (0-1 scale)
        feature_frequencies = {feature: count/total_selections 
                             for feature, count in feature_counter.items()}
        
        # Jaccard Stability Index: Average pairwise Jaccard similarity
        jaccard_scores = []
        for i in range(len(all_selected_features)):
            for j in range(i+1, len(all_selected_features)):
                set1, set2 = all_selected_features[i], all_selected_features[j]
                if len(set1.union(set2)) > 0:
                    jaccard = len(set1.intersection(set2)) / len(set1.union(set2))
                    jaccard_scores.append(jaccard)
        
        stability_score = np.mean(jaccard_scores) if jaccard_scores else 0
        
        return stability_score, feature_frequencies
    
    def pre_filter_features(self, X, y, method='mutual_info', top_k=50):
        """Step 1: Pre-filter features using statistical methods"""
        print(f"🔍 Step 1: Pre-filtering to top {top_k} features using {method}...")
        
        if method == 'mutual_info':
            selector = SelectKBest(score_func=mutual_info_classif, k=min(top_k, X.shape[1]))
        else:
            selector = SelectKBest(score_func=f_classif, k=min(top_k, X.shape[1]))
            
        X_selected = selector.fit_transform(X, y)
        selected_features = selector.get_support(indices=True)
        
        print(f"✅ Reduced from {X.shape[1]} to {len(selected_features)} features")
        return X.iloc[:, selected_features], selected_features
    
    def create_stability_aware_fitness(self, X_train, y_train, X_val, y_val, 
                                     feature_frequencies=None, stability_weight=0.1):
        """
        SPEED-OPTIMIZED fitness function that considers both accuracy AND feature stability
        
        SPEED OPTIMIZATIONS:
        1. Reduced CatBoost iterations: 100 → 30 (70% time reduction)
        2. Early stopping enabled
        3. Pre-converted numpy arrays for faster indexing
        4. Cached model parameters
        """
        # Pre-convert to numpy for faster indexing (major speed boost)
        X_train_np = X_train.values
        X_val_np = X_val.values
        y_train_np = y_train.values
        y_val_np = y_val.values
        
        def fitness_function(solution):
            selected_indices = np.where(np.array(solution[:X_train.shape[1]]) > 0.5)[0]
            
            if len(selected_indices) == 0:
                return 1e6  # Penalty for no features
            
            # Quick feature count check (early termination for too many features)
            if len(selected_indices) > self.target_features * 2:  # If more than 2x target
                feature_penalty = 0.5 * (len(selected_indices) - self.target_features)
                return 1.0 + feature_penalty  # Return high fitness without training
            
            # Base fitness: prediction error
            learning_rate = max(0.01, min(0.3, solution[-2]))
            depth = max(3, min(10, int(solution[-1])))
            
            try:
                # SPEED BOOST: Reduced iterations from 100 to 30 (70% faster)
                model = CatBoostClassifier(
                    verbose=0, 
                    learning_rate=learning_rate, 
                    depth=depth,
                    iterations=30,  # MAJOR SPEED IMPROVEMENT
                    random_seed=42,
                    early_stopping_rounds=10,  # Stop early if no improvement
                    use_best_model=True
                )
                
                # Use numpy arrays for faster indexing
                X_train_sel = X_train_np[:, selected_indices]
                X_val_sel = X_val_np[:, selected_indices]
                
                model.fit(X_train_sel, y_train_np, 
                         eval_set=(X_val_sel, y_val_np),  # Enable early stopping
                         verbose=False)
                preds = model.predict(X_val_sel)
                accuracy = accuracy_score(y_val_np, preds)
                
                base_fitness = 1 - accuracy  # Error to minimize
                
                # Feature count penalty (encourage fewer features)
                feature_penalty = 0.1 * max(0, len(selected_indices) - self.target_features)
                
                # Stability bonus (encourage previously stable features)
                stability_bonus = 0
                if feature_frequencies is not None:
                    avg_frequency = np.mean([feature_frequencies.get(idx, 0) 
                                           for idx in selected_indices])
                    stability_bonus = -stability_weight * avg_frequency  # Negative = bonus
                
                total_fitness = base_fitness + feature_penalty + stability_bonus
                return total_fitness
                
            except Exception:
                return 1e6
        
        return fitness_function
    
    def run_single_stability_experiment(self, X_filtered, y, pre_selected_indices, 
                                      feature_frequencies=None, run_number=1):
        """SPEED-OPTIMIZED: Run one complete feature selection experiment with cross-validation"""
        print(f"\n🧪 Stability Run {run_number} [SPEED-OPTIMIZED]...")
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42 + run_number)
        
        run_selected_features = []
        run_accuracies = []
        
        # Pre-convert to numpy arrays once (major speed boost for all folds)
        X_filtered_np = X_filtered.values
        y_np = y.values
        
        for fold, (train_idx, test_idx) in enumerate(kf.split(X_filtered, y), 1):
            print(f"      Processing Fold {fold}/5...", end=" ")
            fold_start_time = pd.Timestamp.now()
            
            # Use numpy indexing for faster data splitting
            X_train_full = X_filtered.iloc[train_idx]
            y_train_full = y.iloc[train_idx]
            X_test_fold = X_filtered.iloc[test_idx]
            y_test_fold = y.iloc[test_idx]
            
            # Further split training data for WOA optimization
            X_train, X_val, y_train, y_val = train_test_split(
                X_train_full, y_train_full, test_size=0.3, 
                random_state=42, stratify=y_train_full
            )
            
            # Create stability-aware fitness function
            fitness_func = self.create_stability_aware_fitness(
                X_train, y_train, X_val, y_val, 
                feature_frequencies=feature_frequencies,
                stability_weight=0.15  # 15% weight for stability
            )
            
            # Setup optimization problem
            dim = X_filtered.shape[1] + 2
            lb = [0] * X_filtered.shape[1] + [0.01, 3]
            ub = [0.6] * X_filtered.shape[1] + [0.3, 10]  # Lower threshold for features
            
            problem = Problem(fit_func=fitness_func, lb=lb, ub=ub, 
                            minmax="min", verbose=False)
            
            # Run optimization (keeping original epoch=15, pop_size=25 as requested)
            optimizer = OriginalWOA(epoch=15, pop_size=25)
            best_solution, best_fitness = optimizer.solve(problem)
            
            # Extract selected features
            selected_mask = np.array(best_solution[:X_filtered.shape[1]]) > 0.5
            selected_indices = np.where(selected_mask)[0]
            original_indices = pre_selected_indices[selected_indices]
            
            # Evaluate fold performance with speed optimization
            if len(selected_indices) > 0:
                fold_model = CatBoostClassifier(
                    learning_rate=best_solution[-2], depth=int(best_solution[-1]),
                    verbose=0, 
                    iterations=50,  # Reduced from 100 to 50 for final evaluation
                    random_seed=42,
                    early_stopping_rounds=15
                )
                
                # Use numpy arrays for faster training
                X_train_sel = X_train_full.iloc[:, selected_indices]
                X_test_sel = X_test_fold.iloc[:, selected_indices]
                
                fold_model.fit(X_train_sel, y_train_full)
                fold_preds = fold_model.predict(X_test_sel)
                fold_acc = accuracy_score(y_test_fold, fold_preds)
                
                run_selected_features.append(set(original_indices))
                run_accuracies.append(fold_acc)
                
                fold_time = (pd.Timestamp.now() - fold_start_time).total_seconds()
                print(f"{len(selected_indices)} features, Acc: {fold_acc:.3f}, Time: {fold_time:.1f}s")
            else:
                fold_time = (pd.Timestamp.now() - fold_start_time).total_seconds()
                print(f"No features selected, Time: {fold_time:.1f}s")
        
        avg_accuracy = np.mean(run_accuracies) if run_accuracies else 0
        print(f"   Run {run_number} Average Accuracy: {avg_accuracy:.4f}")
        
        return run_selected_features, avg_accuracy

    

    def run_full_stability_selection(self, X_filtered, y, pre_selected_indices,
                                     n_repeats=10, min_frequency=0.6):
        """Run multiple stability experiments with bias and final fine-tuning."""
        all_selected_sets = []
        all_accuracies = []
        feature_names = X_filtered.columns.tolist()
        feature_frequencies = None
        stable_feature_indices = None

        for run_number in range(1, n_repeats + 1):
            # Fine-tuning stage for last two runs
            if run_number > n_repeats - 2 and stable_feature_indices is not None:
                run_selected_features, avg_acc = self.run_single_stability_experiment(
                    X_filtered, y, pre_selected_indices,
                    feature_frequencies=None,
                    run_number=run_number,
                    only_use_features=stable_feature_indices
                )
            else:
                run_selected_features, avg_acc = self.run_single_stability_experiment(
                    X_filtered, y, pre_selected_indices,
                    feature_frequencies=feature_frequencies,
                    run_number=run_number
                )

            all_selected_sets.extend(run_selected_features)
            all_accuracies.append(avg_acc)

            # Update frequency counts
            flat_features = [f for run_set in all_selected_sets for f in run_set]
            counts = Counter(flat_features)
            feature_frequencies = {feat: counts[feat] / len(all_selected_sets) 
                                   for feat in counts}

            # Update stable features set
            freq_df_temp = pd.DataFrame({
                "Feature Index": list(counts.keys()),
                "Frequency": list(counts.values())
            })
            freq_df_temp["Frequency (%)"] = freq_df_temp["Frequency"] / (run_number * 5)
            stable_feature_indices = freq_df_temp[
                freq_df_temp["Frequency (%)"] >= min_frequency
            ]["Feature Index"].tolist()

        # Final stability frequency table
        flat_features = [f for run_set in all_selected_sets for f in run_set]
        counts = Counter(flat_features)
        freq_df = pd.DataFrame({
            "Feature Index": list(counts.keys()),
            "Frequency": list(counts.values())
        })
        freq_df["Feature Name"] = freq_df["Feature Index"].apply(lambda idx: feature_names[idx])
        freq_df["Frequency (%)"] = freq_df["Frequency"] / (n_repeats * 5)

        # Final stable features
        stable_features_df = freq_df[freq_df["Frequency (%)"] >= min_frequency]

        print("\n🏆 Stable Features (≥ {:.0%} frequency):".format(min_frequency))
        print(stable_features_df[["Feature Name", "Frequency (%)"]])
        print("\nAverage Accuracy across runs: {:.4f}".format(np.mean(all_accuracies)))

        return stable_features_df, freq_df
    
    def run_stability_analysis(self, X, y):
        """
        Main method: Run multiple stability experiments and analyze results
        
        This is the complete procedure:
        1. Pre-filter features to reduce search space
        2. Run multiple independent feature selection experiments
        3. Calculate stability metrics across all runs
        4. Select final features based on both performance and stability
        """
        print("🚀 Starting Comprehensive Feature Stability Analysis...")
        print(f"Target: {self.target_features} features across {self.stability_runs} stability runs")
        
        # Step 1: Pre-filter features
        X_filtered, pre_selected_indices = self.pre_filter_features(
            X, y, method='mutual_info', top_k=50
        )
        
        # Step 2: Multiple stability runs
        all_experiments_features = []
        all_experiments_accuracies = []
        
        # First run without stability bias (baseline)
        print("\n" + "="*60)
        print("PHASE 1: Baseline Feature Selection (No Stability Bias)")
        print("="*60)
        
        run_features, run_acc = self.run_single_stability_experiment(
            X_filtered, y, pre_selected_indices, feature_frequencies=None, run_number=1
        )
        all_experiments_features.extend(run_features)
        all_experiments_accuracies.append(run_acc)
        
        # Calculate initial stability metrics
        if len(all_experiments_features) > 1:
            stability_score, feature_frequencies = self.calculate_stability_metrics(all_experiments_features)
            print(f"\n📊 After Run 1 - Stability Score: {stability_score:.3f}")
        else:
            feature_frequencies = None
        
        # Subsequent runs with stability awareness
        print("\n" + "="*60)
        print("PHASE 2: Stability-Aware Feature Selection")
        print("="*60)
        
        for run in range(2, self.stability_runs + 1):
            run_features, run_acc = self.run_single_stability_experiment(
                X_filtered, y, pre_selected_indices, 
                feature_frequencies=feature_frequencies, run_number=run
            )
        

            all_experiments_features.extend(run_features)
            all_experiments_accuracies.append(run_acc)
            
            # Update stability metrics
            stability_score, feature_frequencies = self.calculate_stability_metrics(all_experiments_features)
            print(f"📊 After Run {run} - Stability Score: {stability_score:.3f}")
        
        # Step 3: Final stability analysis
        print("\n" + "="*60)
        print("PHASE 3: Final Stability Analysis & Feature Selection")
        print("="*60)
        
        final_stability, final_frequencies = self.calculate_stability_metrics(all_experiments_features)
        
        print(f"\n🎯 FINAL RESULTS:")
        print(f"Overall Stability Score: {final_stability:.4f}")
        print(f"Average Accuracy Across Runs: {np.mean(all_experiments_accuracies):.4f} ± {np.std(all_experiments_accuracies):.4f}")
        
        # Step 4: Select most stable features
        stable_features = self.select_stable_features(final_frequencies, X.columns,min_frequency=0.2)
        
        return stable_features, final_stability, all_experiments_accuracies
    
    def select_stable_features(self, feature_frequencies, feature_names, min_frequency=0.3):
        """
        Select final features based on stability criteria
        
        Args:
            min_frequency: Minimum frequency for a feature to be considered stable
        """
        print(f"\n🔍 Feature Stability Analysis:")
        print(f"Minimum frequency threshold: {min_frequency}")
        
        # Sort features by frequency
        sorted_features = sorted(feature_frequencies.items(), 
                               key=lambda x: x[1], reverse=True)
        
        # Select features above threshold, up to target number
        stable_features = []
        print(f"\n📊 Feature Stability Rankings:")
        print("Rank | Feature | Frequency | Status")
        print("-" * 50)
        
        for i, (feature_idx, frequency) in enumerate(sorted_features, 1):
            feature_name = feature_names[feature_idx]
            status = "SELECTED" if (frequency >= min_frequency and 
                                  len(stable_features) < self.target_features) else "EXCLUDED"
            
            if status == "SELECTED":
                stable_features.append(feature_idx)
            
            print(f"{i:4d} | {feature_name[:20]:20s} | {frequency:8.3f} | {status}")
            
            if i >= 30:  # Show top 30 for brevity
                remaining = len(sorted_features) - 30
                if remaining > 0:
                    print(f"... and {remaining} more features")
                break
        
        print(f"\n✅ Selected {len(stable_features)} stable features")
        return stable_features
    
    def evaluate_final_model(self, X, y, selected_features):
        """SPEED-OPTIMIZED: Train and evaluate final model with selected stable features"""
        print(f"\n🏆 Final Model Evaluation with {len(selected_features)} Stable Features:")
        
        X_final = X.iloc[:, selected_features]
        feature_names = X.columns[selected_features].tolist()
        
        print("Selected Features:")
        for i, name in enumerate(feature_names, 1):
            print(f"{i:2d}. {name}")
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X_final, y, test_size=0.3, random_state=42, stratify=y
        )
        
        # SPEED-OPTIMIZED: Train final model with reduced iterations
        final_model = CatBoostClassifier(
            learning_rate=0.1, 
            depth=6, 
            verbose=0, 
            iterations=100,  # Reduced from 200 to 100 for speed
            random_seed=42,
            early_stopping_rounds=20,  # Add early stopping
            use_best_model=True
        )
        
        # Use eval_set for early stopping
        final_model.fit(X_train, y_train, 
                       eval_set=(X_test, y_test),
                       verbose=False)
        
        # Evaluate
        train_preds = final_model.predict(X_train)
        test_preds = final_model.predict(X_test)
        
        train_acc = accuracy_score(y_train, train_preds)
        test_acc = accuracy_score(y_test, test_preds)
        
        print(f"\n📈 Final Model Performance:")
        print(f"Training Accuracy: {train_acc:.4f}")
        print(f"Test Accuracy: {test_acc:.4f}")
        print(f"Generalization: {test_acc - train_acc:.4f} (closer to 0 is better)")
        
        # Feature importance
        importance = final_model.get_feature_importance()
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importance
        }).sort_values('Importance', ascending=False)
        
        print(f"\n🏅 Top 10 Most Important Features:")
        print(importance_df.head(10).to_string(index=False))
        
        return final_model, test_acc, importance_df

# Cell 3: Main function
def main():
    """
    COMPLETE PROCEDURE EXPLANATION:
    
    1. LOAD DATA: Read your dataset
    2. PRE-FILTERING: Use statistical methods to reduce feature space
    3. STABILITY RUNS: Run feature selection multiple times with different random seeds
    4. STABILITY MEASUREMENT: Calculate how consistently features are selected
    5. STABLE FEATURE SELECTION: Choose features that appear frequently across runs
    6. FINAL EVALUATION: Train final model and report performance
    
    WHY THIS MATTERS FOR YOUR THESIS:
    - Stable features are more likely to generalize to new data
    - Reviewers will appreciate the robustness of your feature selection
    - You can confidently claim your features are reliable, not just lucky
    """
    
    print("🎓 STABLE FEATURE SELECTION FOR THESIS RESEARCH")
    print("=" * 60)
    
    # Load your data
    df = pd.read_csv(".././../data/processed/cleaned_data.csv")
    X = df.drop(columns=["TYPE"])
    y = df["TYPE"]
    
    print(f"📊 Dataset Info:")
    print(f"Samples: {X.shape[0]}, Original Features: {X.shape[1]}")
    print(f"Target Classes: {y.value_counts().to_dict()}")
    
    # Initialize stable feature selector
    selector = StableFeatureSelector(target_features=10, stability_runs=5)  # 3 runs for demo
    
    # Run complete stability analysis
    stable_features, stability_score, accuracies = selector.run_stability_analysis(X, y)
    
    # Evaluate final model
    final_model, test_accuracy, importance_df = selector.evaluate_final_model(X, y, stable_features)
    
    # Summary for thesis
    print("\n" + "="*60)
    print("📝 THESIS SUMMARY")
    print("="*60)
    print(f"✅ Selected Features: {len(stable_features)}")
    print(f"✅ Feature Stability Score: {stability_score:.4f} (0-1 scale, higher is better)")
    print(f"✅ Cross-validation Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"✅ Final Test Accuracy: {test_accuracy:.4f}")
    print(f"✅ Method: Stability-aware Whale Optimization Algorithm")
    
    # What to report in your thesis
    print(f"\n📖 FOR YOUR THESIS PAPER:")
    print(f"1. Method: 'We used a stability-aware feature selection approach'")
    print(f"2. Stability: 'Features were selected based on consistency across {selector.stability_runs} independent runs'")
    print(f"3. Results: '{len(stable_features)} stable features achieved {test_accuracy:.1%} accuracy'")
    print(f"4. Robustness: 'Stability score of {stability_score:.3f} indicates reliable feature selection'")
    
    return stable_features, final_model, stability_score
# Cell 4: Execute main function (if running as script)
if __name__ == "__main__":
    selected_features, model, stability = main()